Chapter 1. Basic Expressions
=======

Expressions in **Prove-It** are fundamental and versatile.  In order to prove some statement to be true, you must be able to express the statement that you want to prove, and express each axiom and each theorem that is used to construct the proof as well as intermediate statements along the way.  
**Expressions** form the basis for **known truths** that are used in the derivation steps of a **proof** and will be discussed later.
An expression is a tree-like data structure (technically a directed acyclic graph, DAG, since there may be multiple instances of the same sub-expression).  As a Python object, its base class is `proveit._core_.expression.expr.Expression` (also aliased as `proveit.Expression` via "`from ... import ...`" Python statements within `__init__.py` files):

In [1]:
from proveit import Expression
%begin basic_expressions
Expression

proveit._core_.expression.expr.Expression

Each `Expression` object has a class that is derived one of the "core" `Expression` classes listed below (each of which is derived from `proveit._core_.expression.expr.Expression`).  The list is reproduced from our <a class="ProveItLink" href="../guide.ipynb">Guide to the Prove-It Library of Proofs</a> but more details and some code examples are provided below in this tutorial.  These "core" types of expressions, in the order that we will be discussing the below, are:
* <a href="#Variable">`Variable`</a>: A label that is interchangeable (as long as it is kept distinct from different labels) with no intrinsic meaning.  It is often represented by a single letter ($a$, $b$, $x$, $y$, etc.) but can have any representation.
* <a href="#Literal">`Literal`</a>: A label that is not interchangeable and has an intrinsic meaning.  Specific operators ($\lnot, \land, +, \times$, etc.) and specific irreducibe values ($\top, \bot, 0, 5$, etc.) are all `Literal`s.  Furthermore, a problem-story `Variable` in a particular **context**, representing some unknown but particular value, should also be a `Literal` (e.g., "Ann has $a$ apples...").
* <a href="#ExprTuple">`ExprTuple`</a>: A list of `Expression`s that is used, for example, when there are multiple *operators* or *operands* of an `Operation`, or multiple *parameters* of a `Lambda`, or multiple *conditions* and corresponding *values* of a `Conditional`.
* <a href="#Operation">`Operation`</a>: The application of *operator(s)* on *operand(s)*.  For example, $0 + 5 + 8$ and $1 < a \leq b < 3$ are examples of *operation* expressions.  The **Prove-It** library defines many *types* derived from the *operation* type (e.g., for each specific operation), but all *operations* have the same behavior with respect to substitution rules that will be described below.
* <a href="#Conditional">`Conditional`</a>: An expression that has one or more *conditions* with associated *values*.  For example, consider this definition for the absolute value of an integer: $\{x~|~x \geq 0; -x~|~x < 0\}$ A *value* is only pertinent if its associated *condition* is satisfied (a **known truth**).  Via axioms, a `Conditional` equates to one of its *values* if its associated *condition* is proven the be true and all of its other *conditions* are proven to false.  With respect to substitution rules, the substitution of a *value* may utilize the fact that its *condition* must be satisfied in order to be pertinent, and therefore its *condition* may be assumed within the context of performing substution for the *value*.
* <a href="#Lambda">`Lambda`</a>: A mapping defined by *parameter* `Variable`s (and/or *iterations* of *indexed variables* described below with the `Iter` and `IndexedVar` core classes) transforming to some *body* `Expression`.  For example, $(x, y, z) \mapsto \{x+y/z~|~(x \in \mathbb{R}) \land (y \in \mathbb{R}) \land (z \in \mathbb{R}) \land (z \neq 0)\}$ is a `Lambda` with a `Conditional` *body* that converts three real numbers $x, y, z$ to $x + y/z$ as long as $z$ is not zero.  Note that a `Lambda` introduces `Variables` into a new scope via the *parameters*.  These *parameter* `Variable`s are said to be *bound* in this new scope; occurrences outside this scope are not deemed to be the same thing. There is a special category of `Operation` types called `OperationOverInstances` that are essentially functionals acting on a `Lambda` operand.  $\forall$, $\exists$, $\sum$, and $\prod$ are examples of `OperationOverInstances`.  For example, $\forall_{x~|~Q(x)} P(x)$ is an expression that translates to "$P(x)$ is true for all values of $x$ for which $Q(x)$ is also true".  Internally, this is represented as an $\forall$ operator acting on the `Lambda` map with a `Conditional` *body*: $x \mapsto \{P(x)~|~Q(x)\}$.
* <a href="#NamedExprs">`NamedExprs`</a>: A mapping from keyword strings to `Expression`s.  This can be used to prevent ambiguity of an expression's internal representation.
* <a href="#Iter">`Iter`</a>: Represents an iteration of a *parameter* going from a *start* index to an *end* index in successive unit increments ($+1$).  It has a `Lambda` map with a `Conditional` body as its sole sub-expression.
For example, $1/(x+i) + ... + 1/(x+j)$ is essentially an `Operation` with `+` as the *operator* and an `Iter` as the *operand*.  This `Iter` in this case is $1/(x+i), ..., 1/(x+j)$.  The sub-expression of this `Iter` may be denoted as $k \mapsto \{1/(x+k)~|~k \in i .. j\}$ where $k$ is an arbitrary `Lambda` *parameter* and $i .. j$ denotes an integer interval. 
* <a href="#IndexedVar">`IndexedVar`</a>: A special kind of `Operation` that indexes a `Variable`.  It treats the `Variable` being indexed as the *operator* and the *index* as the *operand*.  For example: $a_1$, $x_i$, $x_{i+1}$, etc.  An *iteration* if *indexed* *variables* acts as a collection of labels, however.  For example: $a_1, ..., a_n$ is a collection of $n$ labels and can be used as *parameters* of a `Lambda` map and has special substitution rules in which an `Iter` may be expanded to multiple entries within an `ExprTuple`.

Developers and users may derive new classes from core `Expression` classes (`Literal` and `Operation` classes in order to make their own `Expression` classes that have special formatting rules for displaying the **expression** and methods for manipulating and utilizing the expression (i.e., applying theorems or axioms as will be discussed in later tutorial chapters).  Ideally, the formatting should be in direct correspondence with the structure of the `Expression` as a true reflection of the internal representation, but there is no enforcement mechanism to ensure that this is the case.  Users are advised to inspect the fully explicit expression DAGs of important axioms and theorems (or rely on crowdsource checking).  Conveniently, clicking on any **expression** rendered as LaTeX will link to a notebook that, when executed, will reveal this internal structure.  (Alternatively, the `exprInfo()` method may be called).

The remainder of this tutorial chapter will show examples and discuss details of the different core `Expression` classes and discuss their distinct behaviors with respect to *substitution* as well as *meaning* versus *style* equivalence.

The following are related but distinct manipulations in **Prove-It**:
* *Expression substutition*: creating a new **expression** from an existing **expression** by swapping one or more sub-expression(s) for other sub-expression(s) by using the `substituted` method.  This creates a new `Expression` but has nothing to do, intrinsically, with proving statements.
* *Instantiation* derivation step: deriving a new **known truth** from an existing **known truth** via *expression substitution* using specific rules and limitations to ensure that the derivation is sound.  This will be discussed in a later tutorial chapter.
* *Equality substitution*: using the `substitution` axiom of `proveit.logic.equality` to prove that $f(x) = f(y)$ given $x = y$.  This uses *instantiation* of the `substitution` axiom which states $\forall_{f, x, y~|~x=y}~(f(x) = f(y))$.  This specifically involves statements (**known truths**) with the `proveit.logic.equality.equals._operator_` **Literal** (the `=` sign). This will be discussed in a later tutorial chapter.

The focus here will be *expression substitution* with some mention of the additional restrictions applicable to the *instantiation* derivation step before delving into more details in later chapters.  It is important to understand the distinctions among these types of manipulations.

<a name="#Label"></a>Labels (Variables, IndexedVars, and Literals)
================================

In [2]:
from proveit import Label

The `Variable` and `Literal` classes both derive from `Label`.  A `Label` is created with a string and a LaTeX format specification to determine how it is displayed.  In a Jupyter notebook, the LaTeX is rendered and presented in the output:

In [3]:
tri = Label(stringFormat='triangle', latexFormat=r'\triangle')

tri: triangle

After importing anything from `proveit` in a Jupyter notebook, whenever assignments are made at the end of an input cell the output shows the name of the assigned Python variable (not to be confused with a **Prove-It** *Variable*) followed by ':' and its rendered value.  In the above input, we have assigned `tri` to a `Label` object we have created which renders as a triangle shape.  For convenience, this Python variable name and shape appear in the output without any additional code.  The string format is presented when the object is converted to a string and is a useful alternate representation (used, for example, in error messages that are displayed as strings).

In [4]:
str(tri)

'triangle'

The breakdown of an **expression** into its DAG structure is displayed by calling `exprInfo()`.  Clicking on the **expression** is also a way to view this information, linking to a page that shows a canonical way to build the **expression** and calling `exprInfo()` on that built **expression**.  Here we see that the `tri` is a `Label` with no sub-expressions.

In [5]:
tri.exprInfo() # the expressions and sub-expressions are numbered with the top level being zero.

,core type,sub-expressions,expression
0,Label,,


`Label` was not one of the core **expression** types listed above because it is not intended to be used except as the base class of `Variable`, `IndexedVar`, and `Literal`.  Notwithstanding, `Label` is displayed as the core type.
Two Label's are regarded to be the same expression if and only if the class and both formats (string and LaTeX), are the same:

In [6]:
assert tri == Label('triangle', r'\triangle') # equal b/c class (Label) is the same and
                                              # the string and LaTex formats are the same
    
assert tri != Label('tri', r'\triangle')      # not equal when class or either format is different
                                              # Class for both is Label, but string format different

Let's make another label to test out `substituted()` method.

In [7]:
sq =  Label(stringFormat='square', latexFormat=r'\Box')

sq: square

Using *expression substitution*, we can change one `Label` into another or into any other `Expression`.  Later we'll see that *instantiation* is more restrictive, only being applicable to `Variable`s and not any `Label`, but for *expression substitution* this is fine.

In [8]:
tri.substituted({tri:sq})

square

Notice that the `substituted()` method is non-mutating, leaving the tri `Label` as it was originally defined:

In [9]:
tri

triangle

<a href="#Variable"></a>Variable
===========
A `Variable` is an interchangeable <a href="#Label">Label</a> with no formal, contextual meaning.

In [10]:
from proveit import Variable
x = Variable('x')  # the string and latex formats
                   # are the same by default

x: x

In [11]:
# different string and latex formats:
omega = Variable(stringFormat='omega', latexFormat=r'\omega') 

omega: omega

Convenient `Variable`s and other `Expression`s are accessible from `_common_.py` modules (generated from `_common_.ipynb` notebooks).  That is, `_common_.py` modules hold common expressions.  For example, from <a class="ProveItLink" href="../packages/proveit/_common_.ipynb">proveit.\_common\_</a> we can import a collection of commonly-used variables:

In [12]:
from proveit._common_ import a, b, c, x, y, z, alpha
from proveit import ExprTuple
ExprTuple(a, b, c, x, y, z, alpha)

(a , b , c , x , y , z , alpha)

We used an <a href="#ExprTuple">ExprTuple</a> to conveniently render all of those `Variable`s as a single `Expression`.

Expression information for a `Variable` is similar to that for a `Label`, but with the different core type and class:

In [13]:
alpha.exprInfo()

,core type,sub-expressions,expression
0,Variable,,


Just as we saw for the generic `Label` `Expression`s, we can use *expression substitution* to exchange one `Variable` for another or for any other `Expression`.  Furthermore, we will see in a later tutorial chapter that `Variable`s have special properties with respect to *instantiation* because they are defined as interchangeable labels.

In [14]:
alpha.substituted({alpha:sq}) # we can substitute a Variable for any other Expression

square

<a name="#Literal"></a>Literal
-------

A `Literal` is another kind of <a href="#Label">Label</a>.  In contrast to `Variable`s, `Literal`s have a formal, contextual meaning.

In [15]:
from proveit import Literal

Below are different scenarios in which a `Literal` could appropriately be used.

### Specific operators

In [16]:
TIMES = Literal(stringFormat='*', latexFormat=r'\times')

TIMES: *

In [17]:
FACTORIAL = Literal('!')

FACTORIAL: !

In [18]:
SUMMATION = Literal('sum', r'\sum')

SUMMATION: sum

We will see in the <a href="#Operation">Operation</a> section that the *operator* of an `Operation`-derived class should be stored as a Python class variable called `_operator_`.  We are using module-level Python variable names just for these examples.

### Constant values

In [19]:
FIVE = Literal('5') # Literal constants are also ALL-CAPS

FIVE: 5

In [20]:
TRUE = Literal('true', r'\top')

TRUE: true

In [21]:
FALSE = Literal('false', r'\bot')

FALSE: false

Such constants will typically be defined as **common expressions** in the appropriate **context**, to be discussed in a later tutorial.

### Contextual "variables"

This is appropriate when variables are given a specific meaning within the context of a problem.  For example, consider a math "story problem" where Andrea has $a$ apples and Bill has $b$ bananas.  These are variables in a sense, but here they have contextual meaning so they should be `Literal`s.  However, suppose one wants to prove a general theorem for any number of Andrea's apples and Bill's bananas.  One may start by using the $a$ and $b$ `Literal`s for convenience for some contextual "lemmas" (as an aside, though, **Prove-It** does not distinguish between lemmas and theorems), but then demote them to `Variable`s in order to make a statement of the form $\forall_{a, b} P(a, b)$ (`Literal`s may not be quantified over).  This is done by using a process called axiom elimination that will be discussed later.  For now, we just note that "variables" with contextual meaning need to be `Literal`s.

In [22]:
aLit = Literal('a')

aLit: a

In [23]:
bLit = Literal('b')

bLit: b

In [24]:
# different because one is a Variable and one is a Literal
assert a != aLit 
# same after converting the Literal to a Variable
assert a == aLit.asVariable() 

### Literal substitution
It is possible to perform *expression substitution* on a `Literal`.

In [25]:
aLit.substituted({aLit:bLit})

b

In [26]:
aLit.substituted({aLit:omega})

omega

As noted earlier, the `substituted()` method is non-mutating, leaving the aLit `Literal` as it was originally defined:

In [27]:
aLit

a

While we are able to replace a `Literal` via *expression substitution*, we may not replace them via *instantiation* because `Literal`s are not interchangeable labels.

### Literal contexts

A `Literal` is distinguished not only by its formatting but also by its **context**.  Axioms and theorems of **Prove-It** are organized via **contexts** which we will discuss later in more detail.  Each `Literal` belongs to a **context** which is typically the one corresponding to the package (directory) in which it is defined.  This is particularly important for *contextual variable* types of `Literal`s that are very *context*-specific.

In [28]:
import proveit.logic as logic

In [29]:
logic.TRUE

TRUE

*If the above output is not the $\top$ symbol when you execute this, you probably need to <a class="ProveItLink" href="tutorial00_introduction.ipynb#build">build or download the Prove-It database</a>.*

In [30]:
# When the context is different, they are not the same.
TRUE == logic.TRUE

False

The detailed expression information reveals the differences:

In [31]:
TRUE.exprInfo(details=True)

0. true
   core type: Literal
   latexFormat: \top
   context: tutorial
   class: <class 'proveit._core_.expression.label.literal.Literal'>
   sub-expressions:

In [32]:
logic.TRUE.exprInfo(details=True)

0. TRUE
   core type: Literal
   latexFormat: \top
   context: proveit.logic.boolean
   class: <class 'proveit.logic.boolean.booleans.TrueLiteral'>
   sub-expressions:

**Expressions** are also distinguished by their class (derived from the *core type*).  In the above example, `logic.TRUE` is an object of the `proveit.logic.boolean.booleans.TrueLiteral` class.  `logic.TRUE` and `logic.FALSE` have an `evalEquality(..)` method defined in their respective classes which is a convenient method for deriving any of the following: $(\top = \top) = \top$, $(\bot = \bot) = \top$, $(\top = \bot) = \bot$, or $(\bot = \top) = \bot$ (these are proven theorems within `proveit.logic`). For example:

In [33]:
logic.TRUE.evalEquality(logic.FALSE)

|- (TRUE = FALSE) = FALSE

That is a sneak preview of a known truth (**KnownTruth** object) that uses the turnstile notation, $\boldsymbol{\vdash}$, to indicate that the expression is a proven statement.  There will be more on this in later tutorial chapters.

<a name="ExprTuple"></a>ExprTuple
=========

An `ExprTuple` is an ordered collection of **expression** objects which we previewed briefly already.  Here is another example:

In [34]:
et = ExprTuple(a, b, c, TRUE, FIVE, a, alpha)

et: (a , b , c , true , 5 , a , alpha)

An `ExprTuple` may be used as the *operands* or *operations* of an `Operation`, the *parameters* of a `Lambda` map, or the *values* and *conditions* of a `Conditional`.  A Python tuple (or any iterable) will automatically be converted to an `ExprTuple` when it is passed in for any of these roles when constructing an `Operation`, `Lambda`, or `Conditional`. 

Substitution typically carries forward in a straightforward manner.

In [35]:
et.substituted({a:FALSE})

(false , b , c , true , 5 , false , alpha)

Note that the *substitution* occurs on all relevant instances.  And, as always, `substituted` is non-mutating:

In [36]:
et

(a , b , c , true , 5 , a , alpha)

The **expression** constituents of an `ExprTuple` are known as *entries*:

In [37]:
et.entries[5]

a

The enumerated mathematical objects represented by the `ExprTuple` are known as *elements*.  The *elements* and *entries* are in correspondence except when the `ExprTuple` contains an `Iter`.  A contained `Iter` is a single *entry* but may represent multiple elements.  For example, the second *entry* of $(a, b_1, ..., b_n)$ is the `Iter` $b_1, ..., b_n$, but the second *element*, assuming $n \geq 1$, is $b_1$.  We will revisit this in the <a href="#Iter">Iter</a> section.

`ExprTuple` *substitution* is more interesting when an `Iter` containing an `IndexedVar` is expanded such that the `Iter` splits into multiple *entries*.  This will be demonstrated in the <a href="#IndexedVar">IndexedVar</a> section.  

<a name="Operation"></a>Operation
=========

This type of **expression** represents an applied operation.  It contains *operator(s)* and *operand(s)*.  The default formatting of an `Operation` with one *operator* and more than one *operand* is to place the *operator* between each successive pair of *operands* as follows.

In [38]:
from proveit import Operation
triple_prod = Operation(TIMES, [x, y, omega])

triple_prod: x * y * omega

A `Function` is derived from `Operation`.  It behaves in the same manner except it formats it in a "function" style:

In [39]:
from proveit import Function

# Simple case: a Variable operator with a single Variable operand
f = Variable('f')
fx = Function(f, x)

fx: f(x)

Common forms of these are available from <a class="ProveItLink" href="../packages/proveit/_common_.ipynb">proveit._common_</a>:

In [40]:
from proveit._common_ import fx, gx, Px, Qx
ExprTuple(fx, gx, Px, Qx)

(f(x) , g(x) , P(x) , Q(x))

An `Operation` may have a single *operator* that is a *label* (*variable* or *literal*), or it may have multiple *operators* that are *labels* and/or *iterations of indexed variables.*

In [41]:
try:
    Operation(fx, fx)
    assert False, "Expecting a TypeError; should not make it to this point"
except TypeError as e:
    print("Expected TypeError: %s"%str(e))

Expected TypeError: operator(s) must be a Label, an indexed variable (IndexedVar), or iteration (Iter) over indexedvariables (IndexedVar). f(x) is none of those.


The `operators` and `operands` attributes access the *operators* and *operands* respectively as `ExprTuple`s:

In [42]:
triple_prod.operators

(*)

In [43]:
triple_prod.operands

(x , y , omega)

If there is only one *operator* or *operand*, it may be accessed via the respective `operator` and `operand` attributes: 

In [44]:
triple_prod.operator

*

In [45]:
fx.operand

x

In [46]:
# An Operation does not have an `operand` attribute
# if there is more than one *operand*.
hasattr(triple_prod, 'operand')

False

Looking at the expression information, we see that `triple_prod` has `operands` that is an `ExprTuple` but a single `operator` that is a `Literal`.

In [47]:
triple_prod.exprInfo()

,core type,sub-expressions,expression
0,Operation,operator: 1operands: 2,"<img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAFcAAAAOBAMAAACoZ51gAAAAMFBMVEX///8QEBC6uroyMjJERETc 3NyYmJgiIiJmZmaqqqp2dnbMzMxUVFSIiIju7u4AAAAoIwv1AAAAAXRSTlMAQObYZgAAAAlwSFlz AAAOxAAADsQBlSsOGwAAAQRJREFUKM9jYICDo0AczIAMMEXe3nm9agKIEb6BgbUaLNLLwHcARYTh AITH0MB7lqMAJAAUB/EZGBZMZGD7gCLCCTIt/gIDWwDvV54EsPbwrWBjWCc0M/A4IIswcIAIrgkM 3AwsBlAHsZaDjeFjkGVgvIAswqDEwOAANBbI4r+AqpiB6R/D/g0oIhcYuBsYuEAs/QC+B6iWGjBc ZEARaWDgncAQxcDwROE8RA/COywHGHxQRbwZ9jfw2TEwzN/XyKDJgBJQHAeYvqGKLFwj+acRGAov 7nK2gYMZqB0aBXynbnxFFXnyOeCiLAMOwGXAQCxoZng4gWjFtQyniFbLMPeGAnYJAP35UAHKoGi+ AAAAAElFTkSuQmCC "" style=""display:inline;vertical-align:middle;"" />"
1,Literal,,
2,ExprTuple,"3, 4, 5","<img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAEMAAAAVBAMAAADvM06qAAAAMFBMVEX////MzMxUVFREREQiIiK6 uroQEBCYmJhmZmYyMjLu7u52dnaqqqqIiIjc3NwAAACFAiZ1AAAAAXRSTlMAQObYZgAAAAlwSFlz AAAOxAAADsQBlSsOGwAAAV1JREFUKM+Fkj1LA0EQhp/k8uUld6YUG68SIYjBXo1gqZDCRlC0UQsR UlpJKkMQ4TpBRCIB0SIQbLVIZaOSYCEpFK7wB6TzAwzO7t2158DOPTvz7s3ssIBHhOmk1YmSbCj3 GKXQPzDdSAlnkKlESw4gFa0g1WDtH8lIhWP5tF4u7/x7tU4wHU1xWfYiZB1OBavWdKakE+0H0gNN T0ryC2mXZfGeNYzrru1OnXhRS6rK3UOuxDbEMIKbm+yT6ilKljHgUJIDJYHRXtBc8oeJvB4ZjEld iWxxrgKbntnQkozLqw/wht3WheagWZ4hC0pkOMxqMvKxL24E0kVuoTt+xDrmtzrsJD81Zfcmu1O7 KtJjFa7fczUZy5KcMQv9oSZz4cLaUR0mPBL5cI453U7WDcm3D6lVDTfSDnWuOj6FVpM1H25aslYo BBSYrQbZDHdqvs/9ckCBGaqmHflgZL5/m+xNbhqL7s4AAAAASUVORK5CYII= "" style=""display:inline;vertical-align:middle;"" />"
3,Variable,,
4,Variable,,
5,Variable,,


### Defining an `Operation` sub-class

Let's derive a couple of classes from `Operation` to represent operations with specific literal operators.
This is commonly done within Prove-It library modules.  We use the `%load` magic command to display (and execute) the contents of <a href="demo_operations.py">demo_operations.py</a>.

In [48]:
# %load demo_operations.py
'''
Module that defines a Operations for demonstration
purposes in this tutorial.
'''

from proveit import Operation, Literal

class Factorial(Operation):
    # _operator_ is a special class variable name, defining specific literal operator of the Operation class.
    # It is not only used for default formatting but also when performing substitutions for rebuilding expressions.
    _operator_ = Literal('!')
    
    def __init__(self, operand):
        # creates the Operation with FACTORIAL as the operator and the provided operand as its only operand.
        Operation.__init__(self, Factorial._operator_, operand) # initializes self.operand

    def string(self, **kwargs): # should accept kwargs even when not used (e.g., 'fence')
        # the operand should be fenced (wrapped in parentheses) to prevent ambiguity
        return self.operand.string(fence=True) + Factorial._operator_.string()
    
    def latex(self, **kwargs): # should accept kwargs even when not used (e.g., 'fence')
        # the operand should be fenced (wrapped in parentheses) to prevent ambiguity
        return self.operand.latex(fence=True) + Factorial._operator_.latex() 

class Multiply(Operation):
    
    # This operator Literal has a LaTeX format that differs from the string format.
    _operator_ = Literal('*', r'\times')
    
    def __init__(self, *operands): # takes a list of arguments as the operands
        # creates the AssociativeOperation with TIMES as the operator and any number of operands.
        Operation.__init__(self, Multiply._operator_, operands)
    
    # The default formatting will display the operator between the operands


In order to work properly, however, we need to import these classes from the module (as Prove-It will use the `__file__` attribute of this module for its internal purposes).

In [49]:
from demo_operations import Factorial, Multiply

A `Factorial` is an `Operation` with "!" as the format of the `Literal` operator.  Its formatting is altered from the default but is still true to the core `Expression` structure (the formatting is a reflection of the internal structure).

In [50]:
# Now we can make an object with this new class
x_factorial = Factorial(x)

x_factorial: x!

In [51]:
# show core structure
x_factorial.exprInfo()

,core type,sub-expressions,expression
0,Operation,operator: 1operand: 2,
1,Literal,,
2,Variable,,


In the Jupyter notebook setting, we typically only use LaTeX formatting, but it is important to define the string formatting as well for whenever it is needed (e.g., when error messages are displayed).  The `fence=True` above indicates that parentheses should be used when it could be ambiguous otherwise.  Our example above has no ambiguity, so parentheses are not used.  We will show a case below where the parentheses are required.

Next we consider a multiple *operand* example using the `Multiply` class that we imported from <a href="demo_operations.py">demo_operations.py</a> above.

In [52]:
# Demonstrating an Operation with multiple operands
mult_expr = Multiply(x, FIVE, omega)

mult_expr: x * 5 * omega

In [53]:
# Let's nest Operations and show proper fencing behavior
nested_operation = Factorial(mult_expr)

nested_operation: (x * 5 * omega)!

In [54]:
nested_operation.exprInfo()

,core type,sub-expressions,expression
0,Operation,operator: 1operand: 2,"<img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAGkAAAAVBAMAAAC3TN2zAAAAMFBMVEX////MzMxUVFREREQiIiK6 uroQEBCYmJhmZmYyMjLu7u52dnaqqqqIiIjc3NwAAACFAiZ1AAAAAXRSTlMAQObYZgAAAAlwSFlz AAAOxAAADsQBlSsOGwAAAZRJREFUOMuVkz1Iw0AUx/9t2iRNbMwoCJLJioPo4KCgttDRoYOLg9BF HRzM6CAliiAoQjZRRAou6tLiKA4dBQWDg5PFDB10agsOLi2+u7SaSBzy4L5+9/7v3n0BcBHB+s7p WtisDiU02gqVhAE8hMbcPMmXwzhbo1mEYoeq8p+34SmeApUsZDN00vhvY1vAoA0R0VRiGaKBZT+a oPIRUP2SnqVMJCwcUk+pjw/fMdTQoeX4ZKa+gwCJU2E91UCqimMayXh29uBhcmM2i2YhQB7ZoANI NuImFmlUwpub8UKP5PqpCNkAsVhFCQ3QAdawyq90pueq5fW+Su76SbIAgYVHrAW1wFVA549KsCG3 /SQNDAFVkhchuTjjkbq4DuQjZhFv+YkMvEKr8gzJ5ugIp8mjGti7SsJJPxH02Beu6JFJHN+QvH1Z TNYC56xZeCn7ibox+jS2zhZ1ULGxRGsdOffbPJ2pnzt93z0IEGXhPL3msAfvsltO6IhkDV5LVjTV vtfMRxJpk157EUkl9P6qZkZRlXj9DWKFYb2Z+S5YAAAAAElFTkSuQmCC "" style=""display:inline;vertical-align:middle;"" />"
1,Literal,,
2,Operation,operator: 3operands: 4,"<img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAFYAAAAPBAMAAACM+SX7AAAAMFBMVEX///8QEBC6uroyMjJERETc 3NyYmJgiIiJmZmaqqqp2dnbMzMxUVFSIiIju7u4AAAAoIwv1AAAAAXRSTlMAQObYZgAAAAlwSFlz AAAOxAAADsQBlSsOGwAAAPlJREFUKM9jYMACNjDwBTAQCeSNyx8Qq7b8+zpilTIcYGCgotqjQByM ohYhAgVv77xeNQHECN/AwFoNFjp2t5sBRYQHiEGsBt6zHAUMEA5QEgT8GOIVUERmgjh/GdgCeL/y JECM2VoNs4ylAEWkAUSsZOBmYDGAKmAt3wBTy/EVWYRJgYGFgSEVyOK/gKYWqJnjN7IILwPDDgaG BUCWfgDfAxQbGQsYeD4gi3AwMNxkYF3A8EThPAMXA4pPgFxGB2QRlg3cPxgeP2CYv6+RQRM1hFgb GC4+QBbhkjo4/4QYA8OLu5xt4OBl8IaHfERPH4oIn78pr+AFBgBEnUuADtJypQAAAABJRU5ErkJg gg== "" style=""display:inline;vertical-align:middle;"" />"
3,Literal,,
4,ExprTuple,"5, 6, 7","<img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAEMAAAAVBAMAAADvM06qAAAAMFBMVEX////MzMxUVFREREQiIiK6 uroQEBCYmJhmZmYyMjLu7u52dnaqqqqIiIjc3NwAAACFAiZ1AAAAAXRSTlMAQObYZgAAAAlwSFlz AAAOxAAADsQBlSsOGwAAAW1JREFUKM9jYGB4wIAHgCV5DyALCTBwoWiJBRGnUXTlz3RZgMwHGcA1 AUWJy5ddqDbNYmDgKEARUUB3TAUDAxsDfiVsCxgiCSjhLGDoBlJcd7Wk90BE1O82QhhMQMznyMDA rcAwHcjkYLh0oQUiY8XwPgDMOANS8peBgX0Cgw+QWcdw/4E6zGRmBzDVACKARvM4MKSCw8sSYTnH PxDJGsDADNLLwPgBrISB4S/MiAkMHB/BQcbAIMHAsAGoNoFhDljPP4ZVEC86MDB9AJvFwHCHgW8D 2CJboIdMgcIbGLiAMtxAVQYMwChgFmD8wbASyGA3YNgEVPhxRQLrAYb1wKjga2C4toDrJ1Btttp5 jXSQcRcYQoCm9F44UcPAwOIFFHnR1AWMqAUMXPbzeNMuAAVYHjCwCMD9cgrG4EGK7CdAuxrgvA0w BjdSBLQDsR2Mw3UBxlqLUMFnACQWw3jscHGk9MEMspOvAF/aBYYvAPUPTFlmvxPoAAAAAElFTkSu QmCC "" style=""display:inline;vertical-align:middle;"" />"
5,Variable,,
6,Literal,,
7,Variable,,


### Operand substitution

*Substitution* of *operands* is straightforward in the way it works.

In [55]:
x_factorial.substituted({x:omega})

omega!

Notice that the substituted() method is non-mutating, leaving the **xFactorial** expression as it was originally defined:

In [56]:
x_factorial

x!

We can also substitute an entire expression in for the variable $x$:

In [57]:
nested_operation_from_sub = x_factorial.substituted({x:mult_expr})

nested_operation_from_sub: (x * 5 * omega)!

Internally, however, there is specific machinery required to regenerate these objects and construct them with the proper classes.  In order for this machinery to work, the `_operator_` class attribute must be properly defined (e.g., `Factorial._operator_` and `Multiply._operator_`).  To demonstrate that this is working properly, note that the classes displayed in the detailed expression info from `exprInfo()` are as they should be:

In [58]:
nested_operation_from_sub.exprInfo(details=True)

0. (x * 5 * omega)!
   core type: Operation
   class: <class 'demo_operations.Factorial'>
   operator: 1
   operand: 2
1. !
   core type: Literal
   latexFormat: !
   context: tutorial
   class: <class 'proveit._core_.expression.label.literal.Literal'>
   sub-expressions: 
2. x * 5 * omega
   core type: Operation
   class: <class 'demo_operations.Multiply'>
   operator: 3
   operands: 4
3. *
   core type: Literal
   latexFormat: \times
   context: tutorial
   class: <class 'proveit._core_.expression.label.literal.Literal'>
   sub-expressions: 
4. (x , 5 , omega)
   core type: ExprTuple
   class: <class 'proveit._core_.expression.composite.expr_tuple.ExprTuple'>
   sub-expressions: 5, 6, 7
5. x
   core type: Variable
   latexFormat: x
   class: <class 'proveit._core_.expression.label.var.Variable'>
   sub-expressions: 
6. 5
   core type: Literal
   latexFormat: 5
   context: tutorial
   class: <class 'proveit._core_.expression.label.literal.Literal'>
   sub-expressions: 
7. omega
   core type: Variable
   latexFormat: \omega
   class: <class 'proveit._core_.expression.label.var.Variable'>
   sub-expressions:

Furthermore, this expression that we obtained via substitution is equivalent to the one constructed directly:

In [59]:
print("Expressions, generated in different ways, are the same:", 
      (nested_operation_from_sub == nested_operation))

Expressions, generated in different ways, are the same: True


### Operator substitution

The *operator* itself may likewise be *substituted*, but with some interesting extra capabilities.

It is straightforward to substitute a **Variable** *operator* with another **Variable** (i.e., relabel). Here we relabel the $f$ in $f(x)$ to $g$ to obtain $g(x)$:

In [60]:
from proveit._common_ import g
fx.substituted({f:g}) # Variable operator to a different Variable

g(x)

And it is straightforward to *substitute* a **Variable** *operator* with a **Literal**.

In [61]:
x_factorial_from_fx = fx.substituted({f:FACTORIAL})

x_factorial_from_fx: x!

In [62]:
x_factorial_from_fx.exprInfo(details=True)

0. x!
   core type: Operation
   class: <class 'demo_operations.Factorial'>
   operator: 1
   operand: 2
1. !
   core type: Literal
   latexFormat: !
   context: tutorial
   class: <class 'proveit._core_.expression.label.literal.Literal'>
   sub-expressions: 
2. x
   core type: Variable
   latexFormat: x
   class: <class 'proveit._core_.expression.label.var.Variable'>
   sub-expressions:

In [63]:
assert x_factorial_from_fx == x_factorial # Same

It is able to generate the new **expression** in the appropriate `Factorial` class via the internal mechanisms that use its `_operator_` class attribute.

Performing an *expression substitution* of a `Literal` *operator* within an `Operation` class will typically be blocked because it will not know how to make the new **expression**:

In [64]:
from proveit import OperationError
# But this will typically be prevented (appropriately) in trying to remake a derived Operation class:
try:
    x_factorial.substituted({FACTORIAL:SUMMATION})
    assert False, "Expecting an OperationError error; should not make it to this point"
except OperationError as e:
    print("EXPECTED ERROR:", e)

EXPECTED ERROR: An implicit operator may not be changed


### <a name="OperationSub"></a>Operation substitution

The more interesting and useful case is to *substitute* the `Operation` itself. Here we convert $f(x)$ to $x!$ by substituting a `Lambda` mapping for the function variabe $f$:

In [65]:
from proveit import Lambda
fancy_factorial_map = Lambda(y, Multiply(y, Factorial(y)))

fancy_factorial_map: y -> (y * y!)

The <a href="#Lambda">`Lambda`</a> core type will be discussed in more detail below.  We use it here for a quick example.

In [66]:
fx.substituted({f:fancy_factorial_map})

x * x!

Note that the operand is still $x$ (not $y$).  This is substituting the *operation*, not the operand.  If desired, however, it could be substituted simultaneously:

In [67]:
fancy_omega_fn = fx.substituted({f:fancy_factorial_map, x:omega})

fancy_omega_fn: omega * omega!

In [68]:
fancy_omega_fn.exprInfo(details=True)

0. omega * omega!
   core type: Operation
   class: <class 'demo_operations.Multiply'>
   operator: 1
   operands: 2
1. *
   core type: Literal
   latexFormat: \times
   context: tutorial
   class: <class 'proveit._core_.expression.label.literal.Literal'>
   sub-expressions: 
2. (omega , omega!)
   core type: ExprTuple
   class: <class 'proveit._core_.expression.composite.expr_tuple.ExprTuple'>
   sub-expressions: 5, 3
3. omega!
   core type: Operation
   class: <class 'demo_operations.Factorial'>
   operator: 4
   operand: 5
4. !
   core type: Literal
   latexFormat: !
   context: tutorial
   class: <class 'proveit._core_.expression.label.literal.Literal'>
   sub-expressions: 
5. omega
   core type: Variable
   latexFormat: \omega
   class: <class 'proveit._core_.expression.label.var.Variable'>
   sub-expressions:

Note that by substituting $f$ with a `Lambda` expression, the entire `Operation` is substituted, not just the *operator*.  This is why `Lambda` *operator*s are not allowed (only `Label` operators); otherwise, such a substitution could be ambiguous or confusing (should it substitute the `Operation` or substitute the *operator* with the `Lambda` expression?).  This also highlights the fact that **Prove-It** is not a functional programming or lambda calculus.  Rather, **Prove-It** is designed for manipulating expressions as desired, and *operation substitution* is a useful and powerful, yet relatively straightforward tool for doing so.

<a name="Conditional"></a>Conditional
====

A `Conditional` has one or more *values* associated with one or more *conditions*.  A given *value* is only pertinent if its associated *condition* is satisfied (known to be true).

In [69]:
from proveit import Conditional

To create the `Conditional`, supply the *values* and *conditions* as separate arguments in the constructor.  This example invokes the `Neg`, `Less`, and `GreaterEq`  <a href="#Operation">`Operation`</a>s as well as the `zero` <a href="#Literal">`Literal`</a>)

In [70]:
from proveit.number import zero, Neg, Less, GreaterEq
abs_cond = Conditional((Neg(x), x), 
                       (Less(x, zero), GreaterEq(x, zero)))

abs_cond: {-x | x < 0; x | x >= 0}

In [71]:
# The string formatting is somewhat different:
str(abs_cond)

'{-x | x < 0; x | x >= 0}'

The `conditions` and `values` attributes may be used to access the *conditions* and *values* respectively.

In [72]:
abs_cond.conditions

(x < 0 , x >= 0)

In [73]:
abs_cond.values

(-x , x)

This is the expression information breakdown for our `Conditional` example.

In [74]:
abs_cond.exprInfo()

,core type,sub-expressions,expression
0,Conditional,values: 1conditions: 2,"<img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAHoAAAAyBAMAAABhfNLWAAAAMFBMVEX///+qqqoyMjIiIiIQEBDM zMxUVFRmZma6urqYmJh2dnbu7u6IiIhERETc3NwAAABX2VgGAAAAAXRSTlMAQObYZgAAAAlwSFlz AAAOxAAADsQBlSsOGwAAAgtJREFUSMfFljtLw1AUgE8GbdKkD1wcrQgVB8HHIIiD4C6ZdbAIDqJg oV2kIvUX+Nql4qaCgSqFqlgQXCy0ODnmD4h1kKLWYm6T2pP2pJLcwTP0Jm2+3NNzT/JdACNeyuAU FfgrlG+Ng77YA++0WAUvtP98mw2Biid6ELK6MYTKXmhpDPwMvIu5pPtUlnEMhC9Gx93Rhyn2GYqD +EbTNw9Xp2FEy8W1hWPrt/uzxmBQ4gdNx5V13xiiffBQSJj3OSmYl7xmoKdG0pKu1Hv3EH0EJX2V HQQTzd5g9KeNlocmWag9EIjY/rcKT+bpcrh57V0GxJpD1UKFtqrV2mmjaoJD1WBXlzVMi3W4tmce KINEr9htegP8aG75ubcCGXvVpFnwRUh6YDEJO4gW3nOzopWzfP9oHozDpUrSuaKwGcZzbxX691vd Yi54vrgCHnoNDlR04pq2xVKGhy6leeg54KB98zx0UuWgs3HgoVPwb5lzVg0mvNGWDVx0izzSYQM3 nZqf6bCBm6dEmbZ8680GkoU72KBlAtoGwrDexQYtE1A2YG/IqOZoA2QCwgYNelQjbcACmYCwAcs8 qnezwa8JCBsYuU1pXW3wawLCBmjFSBsgExA2QN1C2gCZoNMGQdSppA2QCRxs0GxaywYxnieUj3ax 45Inze0B+irwxTN3o+E4drlhHlqp8tDed/c/n1Htw++lAx8AAAAASUVORK5CYII= "" style=""display:inline;vertical-align:middle;"" />"
1,ExprTuple,"3, 10","<img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAD0AAAAVBAMAAAAHs/uBAAAAMFBMVEX////MzMxUVFREREQiIiK6 uroQEBCYmJhmZmYyMjLu7u52dnaqqqqIiIjc3NwAAACFAiZ1AAAAAXRSTlMAQObYZgAAAAlwSFlz AAAOxAAADsQBlSsOGwAAARZJREFUKM9jYGB4wIALgGV4D+CUjwURp3FKg7VyTcAtzzCLgYGjAI98 BQMDGx5pBrYFDJH45DkLGLqRuGsvLdwN8Q3XXS3pPQwM3AoM05HkG3h1ORzALA6GSxdaGBjYJzD4 IKTZH/D+Y4I4t47h/gN1BgYeB4ZUoGFTXEBAgJGBGeZZAQZLEMX4ASSPAPwX4My/III1gWEOsnz8 A64FEBbrP4ZVYPNtEbKLA/QZuBkYgCq4TJk+MGwAusiAYRNC/rxUG0MMA9dPoL6PKxJYgT7luMAQ gpBfcZenHSjosoCBq/fCiRqgCMsDBhYB9EDjWQBnPgFa0YAuz41gtgOxHbr8WjiLzwDkanR5RIQz g2ziw50A6hgYAFsYPPBCQBH4AAAAAElFTkSuQmCC "" style=""display:inline;vertical-align:middle;"" />"
2,ExprTuple,"4, 5","<img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAHkAAAAVBAMAAACQ4lxbAAAAMFBMVEX////MzMxUVFREREQiIiK6 uroQEBCYmJhmZmYyMjLu7u52dnaqqqqIiIjc3NwAAACFAiZ1AAAAAXRSTlMAQObYZgAAAAlwSFlz AAAOxAAADsQBlSsOGwAAAfNJREFUOMudlE9oE0EUxn9psk06u4k568F6EgtqUDxYUCL0WMSDl4KS ePC+hyAKUnMoFitKbgXxsNIiVSkN6sla2KPioUvoQYTgHrx4sp78U1ucmWQ3zTTsIQM7vDdvvnnv ffPNAiHDDY3L+0Oir6vp0+CYvfE4Abn+OdRpRWNw/CnLRksv3Ngc88VleAI5dyA4W8Yu9S+Jt0Fk 1uAN3IbRA8CPRTmlx3H+mpGtd13jDGx6jHrMmFtqC2o+VMf6d+DgWhf+Gyo+Yy4PpbPaerbu95+/ JtG/OgW3J468j+vSVAqFdrHHWZRePX8yV+7v7UeT1K62crSCuTj7zIScCjtwNCDbYFoyFOb3RjR5 hbmIQ4Xe0dYsX8PjPfRkhG7ilLkJKdLdazsRK2etidXJXeT8PkZP6QoluhKQ2lZoyVFgoiVrTsTa rsGoZm3Tx6pKVchRCYXXX3m6RLZ7Y9YeL40bOwv3PFX5BVi6chrbUES2Sq6BPFKcG9mmaajlFjyQ e0q8lkUcvs+1SFBbXzrGNM+L4g84P1eqlm7oe0+VGV/ckLcRcBVW2s68byriVfsYTHmIR8GHu6Zq xEZLCjITkimaoTu9FcdLeqLfZPH1hLid+MDn5XcxIb6631meUuNS7BfUG1xKQLtJqdOqrYI75J9p Fv4D8EmICfaE9lwAAAAASUVORK5CYII= "" style=""display:inline;vertical-align:middle;"" />"
3,Operation,operator: 6operand: 10,
4,Operation,operator: 7operands: 9,"<img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAADEAAAAQBAMAAABNQoq8AAAAMFBMVEX///8QEBC6uroyMjJERETc 3NyYmJgiIiJmZmaqqqp2dnbMzMxUVFSIiIju7u4AAAAoIwv1AAAAAXRSTlMAQObYZgAAAAlwSFlz AAAOxAAADsQBlSsOGwAAAMNJREFUGNNjYMAGVt0IwCrOwDyBrxS7TDoDwxJ0MU5rEOnDwDD/AaoE WwVY4AcDg/4EBoa3d16vmgCR4K0CS/CBZBIYGBp4z3IUgCVe1UIUsP5iYNh/gYEtgPcrTwJIIKKM AUlmAQM3A4sBmM9jCbONDyijfwHI4L+AJgNywXyQ1foBfA9QTGNwYWDIf8DwROE8AxcDigsYEhkY moHa9jUyaEJF3tZC/Mc+gU+ageHFXc62CXCfQj209s4G9NBhs2YgDACmnTRipqRnpAAAAABJRU5E rkJggg== "" style=""display:inline;vertical-align:middle;"" />"
5,Operation,operator: 8operands: 9,"<img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAADEAAAARBAMAAACGHlkZAAAAMFBMVEX///8QEBC6uroyMjJERETc 3NyYmJgiIiJmZmaqqqp2dnbMzMxUVFSIiIju7u4AAAAoIwv1AAAAAXRSTlMAQObYZgAAAAlwSFlz AAAOxAAADsQBlSsOGwAAAMNJREFUGNNjYMAGVt0IQOI9ToAzmSfwlSLJ8C2/AGOmMzAsQTHj+joo w4eBYf4DFKl0qNQPBgb9CQwMb++8XjUBKjW5FWwuSAZoawPvWY4CmC7lc0CC9RcDw/4LDGwBvF95 4M5S9oPJLGDgZmAxgFs0+SjYNKCMPsih/AjX9sFdMB9ktX4A3wNUV7swMOQ/YHiicJ6BC82niQwM zUBt+xoZNEHcYETosE/gk2ZgeHGXs20CWoDyrb2zgYEy8LwcBKrhfABLZjoVtQ9p3AAAAABJRU5E rkJggg== "" style=""display:inline;vertical-align:middle;"" />"
6,Literal,,
7,Literal

It is actually common to have only one *condition*, indicating a value that is only defined when the condition is satisfied.

In [75]:
from proveit.logic import Equals, NotEquals
from proveit.number import zero, Mult, frac
div_cancel_cond = Conditional(Equals(Mult(x, frac(y, x)), y),
                              NotEquals(y, zero))

div_cancel_cond: {(x * (y / x)) = y | y != 0}

It will then have singular `condition` and `value` attributes.

In [76]:
div_cancel_cond.condition

y != 0

In [77]:
div_cancel_cond.value

(x * (y / x)) = y

<a name="Lambda"></a>Lambda
====

This type of **expression** represents a mathematical mapping or function.  It contains *parameter(s)* (one or more *variables* and/or *iteration* of *indexed* *variables* that will be discussed in the <a href="#IndexedVar">IndexedVar</a> section below) and a *body* (any **expression** that the *parameters* are to be mapped into).

In [78]:
from proveit import Lambda

In [79]:
# maps any value to 5
map_to_5 = Lambda(x, FIVE)

map_to_5: x -> 5

In [80]:
# Let's look at the Expression info
map_to_5.exprInfo()

,core type,sub-expressions,expression
0,Lambda,parameter: 1body: 2,"<img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAADcAAAAPCAMAAAB3LKe0AAAANlBMVEX///8WFhYNDQ0QEBC6uroy MjJERETc3NyYmJgiIiJmZmaqqqp2dnbMzMxUVFSIiIju7u4AAAANq5mNAAAAAXRSTlMAQObYZgAA AAlwSFlzAAAOxAAADsQBlSsOGwAAAMBJREFUKM+VktkSgzAIRWmzCVmA///ZYnRq0tGZhgc1yoEL V4C1qPtFEqyG0xAQZZlDVCzrGLR/Ezn9zU0yMj1xwtyklpK/M8wTzdzGTH2pYAW5MRDeZ+Y4nqKJ SertabMBWJNVuueA4+/+Ip5OxjC+fr9wCu31hyA9l6X8PBH4cGFHA9JjQr/zIvdcDYNM1781tfTo +x3KQ78wGngkOVtV1irkrGe+hE0qJ//SfmJnCoSYzdwtP/wEp1kXSI0M+wCGPgao+xJPJwAAAABJ RU5ErkJggg== "" style=""display:inline;vertical-align:middle;"" />"
1,Variable,,
2,Literal,,


Here is an example with multiple *parameters* represented by an `ExprTuple` and a `Conditional` *body* .  This maps a pair of real numbers to the first element (invoking the `And`, `Add`, and `InSet` <a href="#Operation">`Operation`</a>s and using the `Reals` <a href="#Literal">`Literal`</a>):

In [81]:
# Can have multiple arguments
from proveit.logic import InSet, And
from proveit.number import Add, Reals
from proveit import Conditional
lambda_expr = Lambda([x, omega], 
                     Conditional(Add(x, omega), 
                                 And(InSet(x, Reals), 
                                     InSet(omega, Reals))))

lambda_expr: (x, omega) -> {x + omega | (x in Reals) and (omega in Reals)}

In [82]:
lambda_expr.exprInfo()

,core type,sub-expressions,expression
0,Lambda,parameters: 5body: 1,"<img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAWwAAAAZCAMAAAAi2eRNAAAANlBMVEX///8WFhYNDQ3MzMxUVFRE REQiIiK6uroQEBCYmJhmZmYyMjLu7u52dnaqqqqIiIjc3NwAAACgirQNAAAAAXRSTlMAQObYZgAA AAlwSFlzAAAOxAAADsQBlSsOGwAAA6lJREFUaN7tmulypCAURplhE7iyvP/Ljlu3LBcEp00qVfFH 0qlKDh/Hy6Yh5ImLCSH88dky+zFuID/1cosR+wzaRB85kOdk+3sc/7FfGvhDpQaw/o7sefrkTcxl u85iUWLiPJjl695h6TpqsbsQO+m77E5sT8JCNjMPyrbzQCLPli4wtQ+06y4PsDvpm+xubP+9jmTz 52TDwI3kxM/bN8r2hNnkNmd9g6EiuaK/ZQ9gDTwnW6ph2VoN69D+yKZ06lZkFkbYl/RT9gAWZXxI tnXDslvLgePL8h8ZPHUcgaakcJxisptdwK/op+yRFWyqlLbz3yBb1oekNA7Q2mMA7IiY1JgglHey EfgV/S1bjkxOESNpz+oB2eAcAzpns2Qum732QIJWZOu5xvPlfLfroNyaoy5mlq2GxiNsJAUCv6Cf suuRkbYihiBpZVRk81L2cmMcc0SLlmz7yuGDq8h+3fmSZzymYzIiiHdRqrjI1j9ICuZgYykQ+AX9 lF2PjLQVMTJVHJcNoRhPagnrgidMtWRrdJ+QyD5OZAjP4EuYUtFmwEYd2PcQAgo2lsKQQXokux4Z aStiiHzNAEQ2NeVMvI4RHifW62ojpmn7dgwdKfc+HMlQ2ZPEeYTuqOXi6UDnoN55aMRi27h1Lmdj KTB4mx7JrkbG2ooYf/+I5ApBFrKVqJw6g7tYILcf6HazZ9KWXfKAVzZnC/A9Ccgp31ODyNlYCgze pCOyMQVlWxFD5Eu0RCpbGnSPIZchRAAasvX7EOWrso2t8URtJ6yp5+VAf00R2mZsNIUgY/Rkzq5G RtqqTiM0XaIbczZwSVjIKraQzek6jPRadVCTzV2N51xFx1KWx+MgonjRFW8yNpoCgTfpsWwsMtR6 jCREl+jGbsQGCnoZINK2ZM/TzCYbmJ5UdZ+9b41QnrHlMWvbAi8dPk4v+t3eOdNymrLxFCW8RU9k I5EhmEqPI0Zy4JL9+2zQS2lYrWxzzgYelvvnpsk1DjWiygPGJZTP5fx6803YfvVcTs6lnc4ZG02R w9v09FCDRBb7Lqhsq7bgafodx3VBG8e8macn6vx5sWi31GJfwnH6OsVgWAu3En7xg6juRzVYeagP PQbqp6+yMex8L+EXP2Il/P7LlMtY99k1+rZ4llhwNxN+sWy4Db9+Vwf/ERynb7JLrLqbEJH95Jsa 4tU9juoo27vsKn3fFnZiVf/AOg27R99B/qhLqWe48dt19iv7Ydm//zeSXU6ISf0Dcn0wEpaPnEsA AAAASUVORK5CYII= "" style=""display:inline;vertical-align:middle;"" />"
1,Conditional,value: 2condition: 3,"<img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAPwAAAAZBAMAAADnDF5lAAAAMFBMVEX///9mZma6urrc3NwiIiKI iIjMzMyYmJgQEBB2dnbu7u5EREQyMjJUVFSqqqoAAABfjFPjAAAAAXRSTlMAQObYZgAAAAlwSFlz AAAOxAAADsQBlSsOGwAAAzRJREFUSMfFlk1oE0EUx182yW67uwkBWz8uUkTxgwp7KGhpxYCrFKya S0EUNeJFEWlKKwGrUkRBrEhuCtIST0JjcalVIS0aSEshYg29FCrIFvQgeCj4carizOx2902aLF1Q Ooed2d23/9/OmzfvDQBEP8Fa2hL8n/Yu6wsf9LDR1vDEbUypfgB84c9Wedc3+HZqMAWwo/JF2Kxi LXVMLHe8BhAS5CZS8IWPZqq9LByAvf3Z1bSvVaXUYFmYz1sziRi+8M+qvjRaQRHLACX7ftjq5OpT U4IZMK6Q38jWwO+sid9UCx/QCKvHxnZafSBVE79BJasQq4H/UhPPRUp4XO908USozgrj8I0m1tfx Eqd1veDgFTK45xcvYHcK01l39pER4nzVmm5ItESbOYXmU2j2C6NkMIPxctfCpWEeTxShJYnwgQyy a88i51/+RjZZfdmOuCLrt2CJYAk5X3xDLYbI2MEH4EyigcNHaSQfTCA8nZ5jV3TXfv/UDzo3yXJO ERZZP4MlVM3F3z9xmP76dYDnTnj0wjltnsMHWGIwET5kIrsCCr002xLROL2SPRBijjmOJVQUeg+v sW8/gzjtPE3CJOsbdP2urh8jo21EGCQN4euaXDvxtq7rR2288pTOUGRWkSSEmctuYYmLxFo3bOfn r9JBN/ThtPizIvQSIMagHirwK3aygWYvxuhSCAN2bpDjNt6VyOGNd54txS4SvSiSv8MIh4/RPNaK 8aqJ7OJ4329sc5xP43IrHcxiiRDGBy4YzPnu2stjkSXIcPhX0B2TSxivJJBdF8bnJCIkGSu/1U4H L7GEtR42XjbSZNCPIj/668mAYHL4R0MPfm9OYTzZWa6d6qT4wihIqTkSaBQh0YuStDYWkph1cr5c hnSELGMj2vdyY2LxJr/2uWXt5B0+7cSRnfxxj1PxijDxwYQQXVGFRr1I/bOdkxgdyzoVT1Mm/wAc 8p10j+BnwmM76a60Fv6bUJK73T2uczUoGPddcno8y2JFQaI1xaPRiuSz4Cqe55dKrReekm3+673s Za5UnttynpJ5/6cd2Odh07vKHSkPa5YLwz7Pev+6vV9f/Hqe8/8COkL9xobuCVIAAAAASUVORK5C YII= "" style=""display:inline;vertical-align:middle;"" />"
2,Operation,operator: 4operands: 5,
3,Operation,operator: 6operands: 7,"<img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAJ0AAAAVBAMAAACu6/FQAAAAMFBMVEX////MzMxUVFREREQiIiK6 uroQEBCYmJhmZmYyMjLu7u52dnaqqqqIiIjc3NwAAACFAiZ1AAAAAXRSTlMAQObYZgAAAAlwSFlz AAAOxAAADsQBlSsOGwAAArNJREFUOMuNVEtoE1EUPZNJOpNJZiwWpChoRPwgiCOtiEHtFIeC4ieI CPWDhfpB3QQqpApK3NiFKFkUhFJjTAtaQSi6cKGLLNyIQoMuBDEwWFHEjQsXCiq++94kM5NMgm8x 79377px3P+deAA7aLuM/NL7FkfRK64Uy1PdraBsQqzbffAjDuTg6MDmaBU6Q8CrMIm6kYuvTrb8n 50MdK3zEobESd00rhBnIxjzMpQy3JOSEG8RceKDmN8hSCpgC1Gw7vFtxFnheyLPuqxNt8VSHmYwD XYELvd/e08CT2eG80L9bK/aRIM5b2y408EwGVsKw/z52p+T5t7HMDreFbPWI+2Byht97/kUesnjj Wdxg4qM3957xBH0tefFKO4rs8NgtQ5xzQWXliBBrBjl3pnzx9rxkFokUdyCvb1It0he9/J07uJ/A L4lyO0meSPY+XhPSH6Ft4H2e/EG+KgXsY19H/xvhZSl4eE8vc+EFF1ml99IeZVFw4Occz1ePHK98 0sJpQIIsgKRTtm3vduNNX6HDKn6xGqjR3pVBLANWp6skLWPWtinw5PtEfek74QFLRB9opo8vvTyc I1zPnpt18XSgF+C0nvH5J+UpV7ERTJN80tF4JSwfnrrSrMdLbF

The `parameters` and `body` attributes access these respective sub-expressions:

In [83]:
lambda_expr.parameters

(x , omega)

In [84]:
lambda_expr.body

{x + omega | (x in Reals) and (omega in Reals)}

If a `Lambda` map has one *parameter*, it will have a `parameter` attribute.

In [85]:
single_param_lambda = Lambda(x, Mult(FIVE, a))

single_param_lambda: x -> (5 * a)

In [86]:
single_param_lambda.parameter

x

### Lambda relabeling and relabeled equivalence

The choice of *parameter* *labels* has no effect on the meaning of a `Lambda` map.  We are free to "relabel" the *parameters* (via `Lambda.relabeled`) as long as it is done consistently and distinct labels remain distinct.

In [87]:
lambda_expr

(x, omega) -> {x + omega | (x in Reals) and (omega in Reals)}

In [88]:
lambda_expr_v2 = lambda_expr.relabeled({x:y})

lambda_expr_v2: (y, omega) -> {y + omega | (y in Reals) and (omega in Reals)}

We can even simultaneously swap labels:

In [89]:
lambda_expr_v3 = lambda_expr.relabeled({x:omega, omega:x})

lambda_expr_v3: (omega, x) -> {omega + x | (omega in Reals) and (x in Reals)}

But we cannot allow a collision of the label mappings:

In [90]:
from proveit import ParameterCollisionError
try:
    lambda_expr.relabeled({x:y, omega:y})
    assert False, ("Expecting a ParameterCollisionError error; "
                   "should not make it to this point.")
except ParameterCollisionError as e:
    print("EXPECTED ERROR: ", e)

EXPECTED ERROR:  Parameter variables must be unique.  [y, y] does not satisfy this criterion.


In [91]:
try:
    lambda_expr.relabeled({x:omega})
    assert False, ("Expecting a ParameterCollisionError error; "
                   "should not make it to this point.")
except ParameterCollisionError as e:
    print("EXPECTED ERROR: ", e)

EXPECTED ERROR:  Parameter variables must be unique.  [omega, omega] does not satisfy this criterion.


Also, an exception is raised if we attempt to relabel to/from a non-Variable.

In [92]:
try:
    lambda_expr.relabeled({x:fx})
    assert False, ("Expecting a TypeError error; "
                   "should not make it to this point.")
except TypeError as e:
    print("EXPECTED ERROR: ", e)

EXPECTED ERROR:  relabel_map must map Variables to Variables. f(x) value is not a Variable.


In [93]:
try:
    lambda_expr.relabeled({fx:x})
    assert False, ("Expecting a TypeError error; "
                   "should not make it to this point.")
except TypeError as e:
    print("EXPECTED ERROR: ", e)

EXPECTED ERROR:  relabel_map must map Variables to Variables. f(x) key is not a Variable.


Moreover, **Prove-It** recognizes the different valid relabeled versions of *Lambda* expressions as the *same* **expression** with respect to *meaning*.  They are regarded as equal.

In [94]:
assert lambda_expr == lambda_expr_v2 == lambda_expr_v3

These are recognized as having distinct *styles* of presentation, however.    **Prove-It** provides flexibility with respect to presentation style of expressions while recognizing the *meaning* equivalence when style is the only distinction.  This is the first of many examples to be encountered.

### Lambda scope and substitution rules

It is important to understand that a `Lambda` expression defines a new scope within its *body* with respect to the *parameters*.  This has two important consequences with respect to substitution rules.

The first consequence of the internal `Lambda` scope is that the `Lambda` masks the substitution of variables that happen to be the same as any of the `Lambda` parameters.  For example,

In [95]:
tuple_with_lambda = ExprTuple(x, lambda_expr)

tuple_with_lambda: (x , (x, omega) -> {x + omega | (x in Reals) and (omega in Reals)})

In [96]:
tuple_with_lambda.substituted({x:aLit}) 

(a , (x, omega) -> {x + omega | (x in Reals) and (omega in Reals)})

Here we see that $x$ is altered external to the `Lambda` expression, but the $x$ within the `Lambda` expression is left alone.  This is fine because the internal $x$ is essentially a different $x$; the scope is different.

For convenience, `substituted` will perform relabeling if the replacement of a `Lambda` *parameter* is another `Variable`.  For example,

In [97]:
tuple_with_lambda.substituted({x:y}) 

(y , (y, omega) -> {y + omega | (y in Reals) and (omega in Reals)})

The inner and outer $y$ *variables* are still regarded as distinct, but it doesn't hurt to relabel the one within the `Lambda` as we wish.  If we do not want this behavior when performing *expression substitution* or *instantiation*, we can *relabel* the map with a distinct variable first.  This feature simply provides a convenient way to perform inner relabeling simultaneously with other substitutions.

The other consequence for of `Lambda` scoping with respect to substitution is that we must not allow any expression within the `Lambda` *body* to be replaced with something that contains one of its *parameters*.  That is because those *parameters* are, with respect to scope, distinct from anything outside of the `Lambda` expression.  So bringing in a replacement defined for external use would have a different meaning when applied internally, and this must not be allowed.  Otherwise, you could alter the meaning of the expression in ways that can invalidate a proof (e.g., via a *instantiation* step which relies upon *expression substitution* internally).  Thus, the parameters of the Lambda function are "reserved" and may not be used in substitution. 

Consider the mapping defined below:

In [98]:
map_to_new_var = Lambda([x, omega], z)

map_to_new_var: (x, omega) -> z

Expressions involving $x$ and/or $w$ are not allowed to be used in the substitutions for $z$. For example, we are not allowed to transform this map to $(x, \omega) \mapsto \omega$ via substitution.

In [99]:
from proveit import ScopingViolation
try:
    map_to_new_var.substituted({z:omega})
    assert False, ("Expecting a ScopingViolation error; "
                   "should not make it to this point")
except ScopingViolation as e:
    print("EXPECTED ERROR: ", e)

EXPECTED ERROR:  Must not make substitution with reserved variables (i.e., parameters of a Lambda function).
These variables are in violation: Scoping violation while substituting(x, omega) -> z.  Must not make substitution with reserved variables (i.e., parameters of a Lambda function).
These variables are in violation: {omega}


And we are not allowed to change the map to $(x, \omega) \mapsto f(x)$ via substitution.

In [100]:
from proveit import ScopingViolation
try:
    map_to_new_var.substituted({z:fx})
    assert False, ("Expecting a ScopingViolation error; "
                   "should not make it to this point")
except ScopingViolation as e:
    print("EXPECTED ERROR: ", e)

EXPECTED ERROR:  Must not make substitution with reserved variables (i.e., parameters of a Lambda function).
These variables are in violation: Scoping violation while substituting(x, omega) -> z.  Must not make substitution with reserved variables (i.e., parameters of a Lambda function).
These variables are in violation: {x}


If it does not involve any of the parameter variables, however, that is okay.

In [101]:
tuple_with_lambda.substituted({x:y}) 

(y , (y, omega) -> {y + omega | (y in Reals) and (omega in Reals)})

In [102]:
map_to_new_var.substituted({z:FIVE})

(x, omega) -> 5

### Applying the Lambda map

In addition to the `substituted` method, the `Lambda` class has an `apply` method.  As arguments, the `apply` method should be given the operands to be mapped.  The `apply` method will apply the proper substitution replacements to the `Lambda` *body* to effect the application of the map to the operands.  Internally, <a href="#OperationSub">*operation substitution*</a> is effected via the `Lambda.apply` method.

In [103]:
lambda_expr

(x, omega) -> {x + omega | (x in Reals) and (omega in Reals)}

In [104]:
lambda_expr.apply(fancy_omega_fn, y)

{(omega * omega!) + y | ((omega * omega!) in Reals) and (y in Reals)}

### Mocking up **Lambda** operators

What if we wanted to represent an expression involving a `Lambda` function applied to operand(s).  We cannot do this using the `Lambda` and `Operation` expression types directly because an `Operation` is only allowed to have *variable* or *iterated indexed variables* as *operators*.   But it would not be consistent with the **Prove-It** philosophy of *freedom of expression* to completely disallow representing such an expression if so desired.  We can, in fact, represent such a thing in **Prove-It** by making an `Operation` class whose *operands* are the `Lambda` function as well as the operand on which we wish to apply the function. As a demonstration, we define such an `Operation`, called `LambdaApplication`, in <a href="demo_lambda_app_operation.py">demo_lambda_app_operation.py</a>.

In [105]:
# %load demo_lambda_app_operation
'''
Module that defines a LambdaApplication Operation
class for demonstration purposes in this tutorial.
'''

from proveit import Operation, Literal, NamedExprs

class LambdaApplication(Operation):
    _operator_ = Literal('LAMBDA_APPLICATION', r'{\rm LAMBDA\_APPLICATION}')
    
    def __init__(self, lambdaFn, operand):
        Operation.__init__(self, LambdaApplication._operator_, NamedExprs([('lambdaFn',lambdaFn), ('operand',operand)]))
        self.lambdaFn = self.operands['lambdaFn'] # The Lambda function operand
        self.lambdaOperand = self.operands['operand'] # The operand of the Lambda function

    @classmethod
    def extractInitArgValue(operationClass, argName, operator, operand):
        '''
        Given a name of one of the arguments of the __init__ method,
        return the corresponding value as determined by the operator and
        operand of the LambdaApplication Operation.
        (This is important so that Prove-It knows how to 'make' an altered
        copy of this Operation).
        '''
        assert isinstance(operand, NamedExprs), "Expecting LambdaApplication operand to be a NamedExprs object"
        if argName=='lambdaFn': 
            return operand['lambdaFn']
        elif argName=='operand':
            return operand['operand']

    def string(self, **kwargs): # should accept kwargs even when not used (e.g., 'fence')
        return self.lambdaFn.string(fence=True) + '(' + self.lambdaOperand.string() + ')'
    
    def latex(self, **kwargs): # should accept kwargs even when not used (e.g., 'fence')
        return self.lambdaFn.latex(fence=True) + '(' + self.lambdaOperand.latex() + ')'


In [106]:
from demo_lambda_app_operation import LambdaApplication

In [107]:
lambdaAppExpr = LambdaApplication(Lambda(y, Factorial(y)), x)

lambdaAppExpr: [y -> y!](x)

Note that the `_operator_` of the `LambdaApplication` is not displayed but is implicit in the context of the lambda application formatting.  Also, we made use of `NamedExprs` which will be discussed next.  Let us take a look at the expression information.

In [108]:
lambdaAppExpr.exprInfo(details=True)

0. [y -> y!](x)
   core type: Operation
   class: <class 'demo_lambda_app_operation.LambdaApplication'>
   operator: 1
   operands: 2
1. LAMBDA_APPLICATION
   core type: Literal
   latexFormat: {\rm LAMBDA\_APPLICATION}
   context: tutorial
   class: <class 'proveit._core_.expression.label.literal.Literal'>
   sub-expressions: 
2. {lambdaFn:[y -> y!], operand:x}
   core type: NamedExprs
   class: <class 'proveit._core_.expression.composite.named_exprs.NamedExprs'>
   lambdaFn: 3
   operand: 4
3. y -> y!
   core type: Lambda
   class: <class 'proveit._core_.expression.lambda_expr.lambda_expr.Lambda'>
   parameter: 7
   body: 5
4. x
   core type: Variable
   latexFormat: x
   class: <class 'proveit._core_.expression.label.var.Variable'>
   sub-expressions: 
5. y!
   core type: Operation
   class: <class 'demo_operations.Factorial'>
   operator: 6
   operand: 7
6. !
   core type: Literal
   latexFormat: !
   context: tutorial
   class: <class 'proveit._core_.expression.label.literal.Literal'>
   sub-expressions: 
7. y
   core type: Variable
   latexFormat: y
   class: <class 'proveit._core_.expression.label.var.Variable'>
   sub-expressions:

**Axioms** could then be created to define how the `LambdaApplication` **operation** actually operates, but **axioms** will be discussed in a later chapter.

NamedExprs
=========

`NamedExprs` is a composite **expression** that maps string keywords to sub-**expressions**.  The reason that this may be desired is to be explicit in the internal representation about the role of each sub-**expression** so there is no ambiguity between the internal and external representation.  A good example is the `lambdaAppExpr` $[y \mapsto y!](x)$ defined above, where one *operand* plays the role of the lambda function and the other *operand* plays the role of the lambda function's operand.

Here is a more rudimentary demonstration of using **NamedExpressions**:

In [109]:
from proveit import NamedExprs
NamedExprs([('left',x), ('right',y), ('product',Multiply(x, y))])

{left:x, right:y, product:(x * y)}

To produce a `NamedExprs` object, you must supply a list of (keyword, `Expression`) pairs.  The keywords must be strings, but can be any string desired.  When the **NamedExprs** is displayed, it will show each of these pairs in the order that they were originally provided.

Iter
=========

It is common in mathematical notation to use ellipses ($...$) in representing an unspecified number of parameters or operands.  The `Iter` expression type serves this purpose.

To create an `Iter`, supply a *parameter*, *body*, and *start* and *end* indices.

In [110]:
from proveit import Iter
from proveit._common_ import i, j, k
from proveit.number import one, Add, frac
it = Iter(k, frac(one, Add(x, k)), i, j)

it: (1 / (x + i)),...,(1 / (x + j))

For compactness, we only show the start and the end instances.  It is implicit that our index parameter ($k$, which is not explicitly shown) is that our index ranges from $i$ to $j$ in increments of $1$.  (Future style options could be implemented, if desired, to make the representation more explicit at the expense of being less compact without changing any of the inner workings of the `Iter` expression type.)

Looking at the expression information, we see that there is just one direct sub-expression of the `Iter` object which is a `Lambda` with a `Conditional` body.  The *condition* of the `Conditional` is that the index is in the integer interval between $i$ and $j$ inclusively. 

In [111]:
it.exprInfo()

,core type,sub-expressions,expression
0,Iter,lambda_map: 1,"<img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAI8AAAAuBAMAAADjDVfrAAAAMFBMVEX///+6urrc3NyYmJiqqqp2 dnYQEBBERESIiIgyMjJmZmZUVFTu7u4iIiLMzMwAAACTvm51AAAAAXRSTlMAQObYZgAAAAlwSFlz AAAOxAAADsQBlSsOGwAAAX9JREFUSMdjYIADvjsMpIG3B7AKn175hyRjeHbNx24QA8cfEl3UP2rQ qEEjySDusr87SDJne/7tBoZRMAqoC/5jBR9IVI9Hw+ADPJhCB8gy6PwEKMHzE5lBOuAFVeC8C4DE LRjjLulO6gJiHwxRFpwG8dXgkLgKxAEYoqw4LeaYgMoXRg5szELrBBqf0QCXwR5IbGZMaXR9LA3E GEQEQDPoRPe5UAcUg3i68JXFCPVwg060g8gFHDu4LqAYxMqYgMcghHq4QQvYgL5mVuD4xG2AYpAK 1wTc5iCphxnErMAPdCIfAztE29q7d9Pu3r0Nygr8eLwGU8919+613Lt3N4AF/cEpDKENFtj+eJMw XD0isNeCSXkFngOoBs3GG1lw9QiDQAn4uMB+eJKFxdoX3GkYWT3cIKaPIG+8WcYggmoQ4wdl3AYh qYcbxAwqH091sCx2QAujCgPcBiGphxvEXUCllM2GHsvCpBkEzbTcDvbklZ3o4L3DFeqU7lxdAkAS AAUCqfVfXUXTAAAAAElFTkSuQmCC "" style=""display:inline;vertical-align:middle;"" />"
1,Lambda,parameter: 17body: 2,"<img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAP8AAAAbCAMAAACEA6lsAAAANlBMVEX///8WFhYNDQ3MzMzc3NyY mJiIiIhUVFR2dna6urqqqqpmZmYyMjIiIiIQEBBERETu7u4AAADpXcuUAAAAAXRSTlMAQObYZgAA AAlwSFlzAAAOxAAADsQBlSsOGwAAAr5JREFUWMPdmNm2pCAMRekmhHn6/5+9WLdwQFAcaujOQy1X CR63CUmAkJdZYGbjbiT/pHFrLTyvFVcbIymi/Wp+TCjq6KSZS40P22PV/fxByuOTQEHjDmPn+YXb G3s/PxoVTkyjvOGqK/zc3McfvLMd4ipmDNS96L/knjf5O7XX/P5O/5seIDE+0IruiClmVvxv9Of5 ozrEf9Q2+aP6OL+MqwWKPuVna17PX9GeXgLew79KptJgeJP/C+2FrNKn+UGYqLGT36T8pFCP2mBC myLgxnMXKIh+n7/QXo60TX7vyRVb5v/IiFLgxnJkqhVbP5MZka5LIznv+Uh07fxfaC+JlW/wD9Nu 41cRpCJ0oq7WVviVBJU/xI4xGHMb1HPcwF9q2zINhRo/NbUQjDvW4teO0UXVdvZpftbEWJkXaMTO rzx2DOCNqvOX2n//2IXFKNf8zMLF7nvBb40WdNEP1VazzklBxd4uUGdvCiMb/i+1bZmIZc3/0uB9 /CF9Yhk3XiKnHD5lyz7LfVIK8kb8r7SX0rRIxOfWP7O4xc/i8DxFpkGIG1VsGNrX/0GOXdHMfyvt JX+ZiA/l/6l4OrnFP5TSkHwwg6qt1swvIwGWvWBaF/nD7tS/lfbiTeX5+j98oskPm/Hv1WNLM+94 AvcytLoYj2PsWd28mL9jm7+iPfcgPciPqEFoKPiZJ1Bs2Xb3v1L4y/2vw57+/+T+19f75fR/Dt5x BEdQwcHV/f8hfq5S+Ic387MhOAkt49+JdINePv9A1z+Wppw2HW81Gqbb+QewB6f2ydzww4blr1cL 6Qx/cAd2qhq1nCayd53/0AQGhf/T8peWwPXzL9BcHZ9FRWvWFf5aMIJLjRYZa4kflz+htijZX3H+ fYW/1oUFzqRAEQr+oYngjP4X/N3n//P8t3/+9QFDax17qYL+av6H/QBgxiBxkShuIwAAAABJRU5E rkJggg== "" style=""display:inline;vertical-align:middle;"" />"
2,Conditional,value: 3condition: 4,"<img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAALUAAAAbBAMAAADIa9MEAAAAMFBMVEX///9mZma6urrc3NwiIiKI iIjMzMyYmJgQEBB2dnbu7u5EREQyMjJUVFSqqqoAAABfjFPjAAAAAXRSTlMAQObYZgAAAAlwSFlz AAAOxAAADsQBlSsOGwAAAjtJREFUSMdjYMAAyw+gcD8wUAD47qFw2UIdqGf26QVodhFt9iIFXDLM j8EUZwMDmWaznluAUy53F4jkPUCM2Vw7HDD0vwcZHYjFYDagwXvBZm8gyt08AhhGTAERntg8dAAq SaTZ/A+wm40LkGK2PixsWbe4+FDZ7BlQLtPRBeS7m3nOxwAsZp9leCoFom0WIGvnCgzANHZpNMxs 7g34kz80nXzne9wOYhxGyMgAEzBTA6YOAY4CBoYIEGt9ATFm833WYwMbi0ixzN8ZGOwYD2BoYDcA xbs9ELMdhQvy/0cFH5DMZmxPgCTdFhcXF28w6wowVy7gxxIm4ITPvOcBQ4kBAzHuZjk4DRI2iBBc eAqWf9CBCCi7HwLFO1Fmr1fQhzgcKRHtBUcxFgDKT+8NcIY3uyuq2fUM6x+AvR+Kksy4/nFiKag+ wdIsZjoB5+4mVLNvM6xPmADO+w+QzWb6YA5yHjBCuf4wQAkG9g848w4DSID5A6rZ+xhYRcFBy3Vb Czl7bAX5mgXkGkcGGAE2Ed1s1kBLYajZ7BtMBbCXsUyzCeV5jgAsZgfzbjgHNZs3+FUzofIbu9m8 D+oXYDG7gDsgAWo2x3SGBEJmR2AVzX/gB820qOHNn8DAuHt39+7tDPaCCQTrnRUPsIkyBoJqOq52 9Honn8EA6u5qJgcDQmYzyz7AJcU2FyzFCi9tmBvmMEyAmn2VzWECFep5eNbiuq4ZuQBq9mGG6wlU MButfQIxm0rtEwas+ZIKZgMAyou5m55fgs8AAAAASUVORK5CYII= "" style=""display:inline;vertical-align:middle;"" />"
3,Operation,operator: 5operands: 6,"<img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAADEAAAAsBAMAAAA++k1qAAAAMFBMVEX///+6urrc3NyYmJiqqqp2 dnYQEBBERESIiIgyMjJmZmZUVFTu7u4iIiLMzMwAAACTvm51AAAAAXRSTlMAQObYZgAAAAlwSFlz AAAOxAAADsQBlSsOGwAAAMRJREFUOMtjYEACbw8wYAU8u+bjkGFg6B+VwS2zPf92A8PwBf9xgA+D 1L3Hc3FKzUIXEIYxvqDLeEBpxj+4ZFgTwNSJ7nOhDqgynBtYuoCJZQHHDq4LqDLnDVzUDBiYFTg+ cRugyvgrOrxUYOBjYJ8A5q69ezft7t3bIOYyO4gK/gZ0F9xc+gBMyyvwHECR4fnI+A8UEgL7GVhR 9TB/4PnCAbTtzTIGEVQZ7gKef0DFpzpYFjugyrA7MHT1YA8DTCCMzAEACQdYHkHzWwQAAAAASUVO RK5CYII= "" style=""display:inline;vertical-align:middle;"" />"
4,Operation,operator: 7operands: 8,"<img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAGcAAAAVBAMAAACphe0AAAAAMFBMVEX////MzMzc3NyYmJiIiIhU VFR2d

The `lambda_map` attributes provides access to the one direct sub-expression.  The `parameter`, `body`, `start_index`, and `end_index` attributes provide convient access to some important indirect sub-expressions.

In [112]:
it.lambda_map

k -> {1 / (x + k) | k in {i...j}}

In [113]:
it.parameter

k

In [114]:
it.body

1 / (x + k)

In [115]:
it.start_index

i

In [116]:
it.end_index

j

We have special formatting features when an `Iter` is used as *operands* of an `Operator`.  As stated previously, **Prove-It** is flexible in its formatting.

In [117]:
add_it = Add(it)

add_it: (1 / (x + i)) +  ...  + (1 / (x + j))

To really understand how **Prove-It** recognizes an expression, one must look at the expression information. 

In [118]:
add_it.exprInfo()

,core type,sub-expressions,expression
0,Operation,operator: 14operands: 1,"<img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAKsAAAAuBAMAAAClu/RBAAAAMFBMVEX///+6urrc3NyYmJiqqqp2 dnYQEBBERESIiIgyMjJmZmZUVFTu7u4iIiLMzMwAAACTvm51AAAAAXRSTlMAQObYZgAAAAlwSFlz AAAOxAAADsQBlSsOGwAAAVpJREFUSMdjYIADvjsMlAAc2k+v/EOJqTi1c1BkLE7to8aOGjtqLI2N 5S77u4MCUynUPogBowEOCWGKjGVpwCHhMWrswBn7Hyv4AJThunv3Wu7duxswtKy9ezft7t3buHWD tY+GLVHG8vyEEecnUNO1t2AE7wIKjOVYQFkgPD+AVZg3gLKippOa5RJO3ZRFhAc9jD3RfS7UAUXi xArizcLUzcDTUQAkF3Ds4LqAIhEgRryxmLoZuBiSGRiYFTg+cRsgSzA5rCTaVEzdDAxGDF+ATX4G 9gmo5RIPww/iOwwYukGpHdxi4G9AC3QWUtohGLqBoQAOFXkFngMoEpwbSDAWQzckNx0X2M/Aimrf eYcAYg3FopuBQV+AgcH/zTIGEVQJeSYDYo3FopuBwR6IT3WwLHZAlWBvJjoIsOhmYNhOm1yWwUCD omYLM2UNWhwp+QuvAy2agysWAgkADECtmMiAC2oAAAAASUVORK5CYII= "" style=""display:inline;vertical-align:middle;"" />"
1,ExprTuple,2,"<img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAKkAAAAyBAMAAADVWuacAAAAMFBMVEX///8yMjKqqqpmZmZERETu 7u6IiIgQEBDMzMzc3NwiIiJ2dnaYmJi6urpUVFQAAABiWEJZAAAAAXRSTlMAQObYZgAAAAlwSFlz AAAOxAAADsQBlSsOGwAAAqVJREFUWMNjYAACvgsM1AIhAXDmG6oZysAhCmPxbUAW7wog0ZylKFwt GOMUkiDrY3nSTI0z/IrCZ34AoTl/oAjnk+hWNlRTGdZAKK4NVDVVHaL/OANVTWU0AFPV1DWV9yc2 UUpNZRAGB+sDKpuqDzKgv4HKpsYnAInnDFQ2lWUBkKijtqk8wOji+ENtU1mBJnL/pLapIN+zFKAK Palfl0CKoXzuv56hCXkxMLA7MFAb7AemAwOqm3r+AkN/AtVNBRqpr0B1U4HeP99AdVP5HzDkT6C+ qQ4M+wOobiq7AIM99U3lWsDwHuLo/1gBbk/+x6OBqwBqKnXdWkCjENiPPw1g2slKTGzl402vrB9h RLwAEoOAqQ8I5YJVIMcuBnnLAMZYSjC9GjDo424PsgENYsIMAlWcGlogiuM3MPQfwB3qwEYNN6bw BpwaEuGlCz9GSciLbE8YZlCjF3J30FUACywujMBnQtaG6RMedAE1dAFg0cJXgNdUIgCGqcCMxQOp zFjTnnZvQjU1zAxfkkOoV4MLQSurEmDN/QnSmGXIS7BCNXXDdTymIqlXgwvVQNrD3+CtjJPA0HiD YiqngiEeU5HUq8GFPsFbGQzmYGYDgyPEwlWrlletWvUAlDO/4TEVpt5q1aryVavWQZwDCUw+UIvo PjQbfEOPLaaveCPpG0ZsMS+AZK0EWFYEN2ZDUU1lfIDPUIR6NeRcA6q4QTkMnAhYffk+wDINzNR4 Bdy5CFk93NT5EF+vg7fgmX4EF/AqoJp6nxN3ZkZWDzf1PFILHuxiVrME5UNo4cqSgye5IqmHm/oE og0ScIwbqJS3yiBehBRcHJV4ShciALR0eQTNpUWoBRiF3eJPXOBw5nuAtXdMLjCzBVOKcAEfWvTk GVioN+oQCI4rAJsV4NEUUWWHAAAAAElFTkSuQmCC "" style=""display:inline;vertical-align:middle;"" />"
2,Iter,lambda_map: 3,"<img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAI8AAAAuBAMAAADjDVfrAAAAMFBMVEX///+6urrc3NyYmJiqqqp2 dnYQEBBERESIiIgyMjJmZmZUVFTu7u4iIiLMzMwAAACTvm51AAAAAXRSTlMAQObYZgAAAAlwSFlz AAAOxAAADsQBlSsOGwAAAX9JREFUSMdjYIADvjsMpIG3B7AKn175hyRjeHbNx24QA8cfEl3UP2rQ qEEjySDusr87SDJne/7tBoZRMAqoC/5jBR9IVI9Hw+ADPJhCB8gy6PwEKMHzE5lBOuAFVeC8C4DE LRjjLulO6gJiHwxRFpwG8dXgkLgKxAEYoqw4LeaYgMoXRg5szELrBBqf0QCXwR5IbGZMaXR9LA3E GEQEQDPoRPe5UAcUg3i68JXFCPVwg060g8gFHDu4LqAYxMqYgMcghHq4QQvYgL5mVuD4xG2AYpAK 1wTc5iCphxnErMAPdCIfAztE29q7d9Pu3r0Nygr8eLwGU8919+613Lt3N4AF/cEpDKENFtj+eJMw XD0isNeCSXkFngOoBs3GG1lw9QiDQAn4uMB+eJKFxdoX3GkYWT3cIKaPIG+8WcYggmoQ4wdl3AYh qYcbxAwqH091sCx2QAujCgPcBiGphxvEXUCllM2GHsvCpBkEzbTcDvbklZ3o4L3DFeqU7lxdAkAS AAUCqfVfXUXTAAAAAElFTkSuQmCC "" style=""display:inline;vertical-align:middle;"" />"
3,Lambda,parameter: 19body: 4,"<img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAP8AAAAbCAMAAACEA6lsAAAANlBMVEX///8WFhYNDQ3MzMzc3NyY mJiIiIhUVFR2dna6urqqqqpmZmYyMjIiIiIQEBBERETu7u4AAADpXcuUAAAAAXRSTlMAQObYZgAA AAlwSFlzAAAOxAAADsQBlSsOGwAAAr5JREFUWMPdmNm2pCAMRekmhHn6/5+9WLdwQFAcaujOQy1X CR63CUmAkJdZYGbjbiT/pHFrLTyvFVcbIymi/Wp+TCjq6KSZS40P22PV/fxByuOTQEHjDmPn+YXb G3s/PxoVTkyjvOGqK/zc3McfvLMd4ipmDNS96L/knjf5O7XX/P5O/5seIDE+0IruiClmVvxv9Of5 ozrEf9Q2+aP6OL+MqwWKPuVna17PX9GeXgLew79KptJgeJP/C+2FrNKn+UGYqLGT36T8pFCP2mBC myLgxnMXKIh+n7/QXo60TX7vyRVb5v/IiFLgxnJkqhVbP5MZka5LIznv+Uh07fxfaC+JlW/wD9Nu 41cRpCJ0oq7WVviVBJU/xI4xGHMb1HPcwF9q2zINhRo/NbUQjDvW4teO0UXVdvZpftbEWJkXaMTO rzx2DOCNqvOX2n//2IXFKNf8zMLF7nvBb40WdNEP1VazzklBxd4uUGdvCiMb/i+1bZmIZc3/0uB9 /CF9Yhk3XiKnHD5lyz7LfVIK8kb8r7SX0rRIxOfWP7O4xc/i8DxFpkGIG1VsGNrX/0GOXdHMfyvt JX+ZiA/l/6l4OrnFP5TSkHwwg6qt1swvIwGWvWBaF/nD7tS/lfbiTeX5+j98oskPm/Hv1WNLM+94 AvcytLoYj2PsWd28mL9jm7+iPfcgPciPqEFoKPiZJ1Bs2Xb3v1L4y/2vw57+/+T+19f75fR/Dt5x BEdQwcHV/f8hfq5S+Ic387MhOAkt49+JdINePv9A1z+Wppw2HW81Gqbb+QewB6f2ydzww4blr1cL 6Qx/cAd2qhq1nCayd53/0AQGhf/T8peWwPXzL9BcHZ9FR

IndexedVar
=========

To truly understand what can be done with `Iter` expressions, we must see how they can be used with `IndexedVar` expressions.  On its own, an `IndexedVar` if fairly straightforward.  An `IndexedVar` is effectively an `Operation` where the *operator* is the variable being indexed and the *operand* is the *index*, but it does have some special properties related to <a href="#IterParams">iterated parameters</a> which justifies makin `IndexedVar` a core expression type.

In [119]:
from proveit import IndexedVar
a_i = IndexedVar(a, i)

a_i: a_{i}

In [120]:
a_i.exprInfo()

,core type,sub-expressions,expression
0,IndexedVar,variable: 1index: 2,
1,Variable,,
2,Variable,,


This "operator" must be a `Variable` or `IndexedVar`:

In [121]:
try:
    IndexedVar(fx, i)
    assert False, ("Expecting a TypeError error; "
                   "should not make it to this point")
except TypeError as e:
    print("EXPECTED ERROR: ", e)

EXPECTED ERROR:  'var' being indexed should be a Variable an IndexedVaritself, got f(x)


By default, a nested `IndexedVar` is formated as having multiple indices:

In [122]:
a_ij = IndexedVar(a_i, j)

a_ij: a_{i,j}

This can be altered by setting the `flatten_nested_indexing` flag to False which changes style options

In [123]:
a_ij_v2 = IndexedVar(a_i, j, flatten_nested_indexing=False)

a_ij_v2: (a_{i})_{j}

This is just a style change, so these two versions are considered to be the same expression with respect to *meaning*.

In [124]:
assert a_ij == a_ij_v2

The real purpose of the `IndexedVar` expression type is to be used within an iteration.  So there is a convenient method called `varIter` for generating an iteration over an indexed variable.

In [125]:
from proveit import varIter
a_iter_i_j = varIter(a, i, j)

a_iter_i_j: a_{i},...,a_{j}

### <a name="IterParams"></a>Iterated parameters

When a `Lambda` expression has iterated parameters, a parameter entry that is an `Iter` over `IndexedVar`s, the `apply` method offers some interesting versatility.  Consider the following example representing a map which distributes a product over a sum of an unspecified number of terms.

In [126]:
a_k = IndexedVar(a, k)
distribution_map = Lambda((a_iter_i_j, b),
                          Add(Iter(k, Mult(a_k, b), i, j)))

distribution_map: (a_{i},...,a_{j}, b) -> ((a_{i} * b) +  ...  + (a_{j} * b))

We can apply a map which splits $a_i, ..., a_j$ into multiple entries.  In so doing, the `Iter` of the result is split in the same manner.  For example, consider the following substitution into $a_i, ..., a_j$:

In [127]:
from proveit._common_ import m, n
x_iter_1_m = varIter(x, one, m)
y_iter_1_n = varIter(y, one, n)
a_sub = ExprTuple(x_iter_1_m, y_iter_1_n, z)

a_sub: (x_{1} ,  ...  , x_{m} , y_{1} ,  ...  , y_{n} , z)

But to allow this as a subtitution, we must prove (or assume) that the lengths of these `ExprTuple`s match (that the have the same number of elements).  We could prove this assuming $j-i+1 = m+n+1$, but for this demonstration we will simply assume what we require to be true.

In [128]:
from proveit.iteration import Len
a_len = Len(Iter(k, k, i, j))
a_required_len = Equals(Len(a_sub), a_len)

a_required_len: |(x_{1} ,  ...  , x_{m} , y_{1} ,  ...  , y_{n} , z)| = |(i ,  ...  , j)|

Using `a_required_len` as an assumption, we can `apply`  `distribution_map` to the operands $(x_1, ..., x_m, y_1, ..., y_n, z)$ to obtain the following

In [129]:
requirements = []
distribution_map.apply(*a_sub, b, assumptions=[a_required_len],
                      requirements=requirements)

(x_{1} * b) +  ...  + (x_{m} * b) + (y_{1} * b) +  ...  + (y_{n} * b) + (z * b)

The `requirements` is appended with facts that were needed to make this happen.  Here, there is just one requirement which is the statement we assumed.  It is a **KnownTruth** proven by *assumption*; these concepts will be discussed in future chapters.

In [130]:
print(len(requirements))
requirements[0]

1


{|(x_{1} ,  ...  , x_{m} , y_{1} ,  ...  , y_{n} , z)| = |(i ,  ...  , j)|} |- |(x_{1} ,  ...  , x_{m} , y_{1} ,  ...  , y_{n} , z)| = |(i ,  ...  , j)|

Recall that the `apply` method is used in the implementation of <a href="#OperationSub">*operation substitution*</a>.  Iterated parameters can be used in that setting as well.  Furthermore, this code is used in *instantiation* of iterated parameters that are universally quantified which will be discussed in a later chapter.

We will remark here that there are limitations in this capability for the sake of simplicity.  We deal with multiple `IndexedVar`s in an iteration, but their subsitutions must be aligned.  For example,

In [131]:
b_k = IndexedVar(b, k)
b_iter_i_j = varIter(b, i, j)
dot_prod_map = Lambda((a_iter_i_j, b_iter_i_j),
                          Add(Iter(k, Mult(a_k, b_k), i, j)))

dot_prod_map: (a_{i},...,a_{j}, b_{i},...,b_{j}) -> ((a_{i} * b_{i}) +  ...  + (a_{j} * b_{j}))

In [132]:
requirements = []
dot_prod_map.apply(*a_sub, *a_sub, assumptions=[a_required_len],
                   requirements=requirements)

(x_{1} * x_{1}) +  ...  + (x_{m} * x_{m}) + (y_{1} * y_{1}) +  ...  + (y_{n} * y_{n}) + (z * z)

As long as the iteration ranges are aligned, we are okay

In [133]:
x_iter_1_n = varIter(x, one, n)
y_iter_1_m = varIter(y, one, m)
b_sub = ExprTuple(y_iter_1_m, x_iter_1_n, z)

b_sub: (y_{1} ,  ...  , y_{m} , x_{1} ,  ...  , x_{n} , z)

In [134]:
b_required_len = Equals(Len(b_sub), a_len)

b_required_len: |(y_{1} ,  ...  , y_{m} , x_{1} ,  ...  , x_{n} , z)| = |(i ,  ...  , j)|

In [135]:
dot_prod_map.apply(*a_sub, *b_sub, assumptions=[a_required_len,
                                               b_required_len],
                   requirements=requirements)

(x_{1} * y_{1}) +  ...  + (x_{m} * y_{m}) + (y_{1} * x_{1}) +  ...  + (y_{n} * x_{n}) + (z * z)

If the ranges are not aligned, this will fail

In [136]:
b_sub = ExprTuple(y_iter_1_n, x_iter_1_m, z)

b_sub: (y_{1} ,  ...  , y_{n} , x_{1} ,  ...  , x_{m} , z)

In [137]:
b_required_len = Equals(Len(b_sub), a_len)

b_required_len: |(y_{1} ,  ...  , y_{n} , x_{1} ,  ...  , x_{m} , z)| = |(i ,  ...  , j)|

In [138]:
from proveit import ImproperSubstitution
try:
    dot_prod_map.apply(*a_sub, *b_sub, assumptions=[a_required_len,
                                                   b_required_len],
                       requirements=requirements)
    assert False, "Expecting a ImproperSubstitution error; should not make it to this point"
except ImproperSubstitution as e:
    print("Expected ImproperSubstitution: %s"%str(e))

Expected ImproperSubstitution: When expanding IndexedVar's within an Iter whose parameter is the index, their expansion Iter indices must all match. (x_{1} ,  ...  , x_{m} , y_{1} ,  ...  , y_{n} , z) vs (y_{1} ,  ...  , y_{n} , x_{1} ,  ...  , x_{m} , z) do not match.


Such cases may still be handled, but the iterations must be first be substituted, via axioms/theorems, to split them apart along the common boundaries.  For example, given the following for the $a$ and $b$ substitutions

In [139]:
a_sub

(x_{1} ,  ...  , x_{m} , y_{1} ,  ...  , y_{n} , z)

In [140]:
b_sub

(y_{1} ,  ...  , y_{n} , x_{1} ,  ...  , x_{m} , z)

If we know that $m$ and $n$ are natural numbers and $m \leq n$, then we can split `a_sub` and `b_sub` respectively into

In [141]:
from proveit.number import subtract
a_sub = ExprTuple(x_iter_1_m, varIter(y, one, subtract(n, m)),
                  Iter(k, IndexedVar(y, Add(subtract(n, m), k)),
                       one, m), z)

a_sub: (x_{1} ,  ...  , x_{m} , y_{1} ,  ...  , y_{n - m} , y_{(n - m) + 1} ,  ...  , y_{(n - m) + m} , z)

In [142]:
b_sub = ExprTuple(y_iter_1_m, Iter(k, IndexedVar(y, Add(m, k)), 
                                   one, subtract(n, m)),
                  x_iter_1_m, z)

b_sub: (y_{1} ,  ...  , y_{m} , y_{m + 1} ,  ...  , y_{m + (n - m)} , x_{1} ,  ...  , x_{m} , z)

Now the `Iter` start and end indices line up: $1$ to $m$, $1$ to $n-m$, and $1$ to $m$, and the last entry is a singular element for both of them.  Now we can apply the map:

In [143]:
a_required_len = Equals(Len(a_sub), a_len)
b_required_len = Equals(Len(b_sub), a_len)
dot_prod_map.apply(*a_sub, *b_sub, assumptions=[a_required_len,
                                               b_required_len],
                   requirements=requirements)

(x_{1} * y_{1}) +  ...  + (x_{m} * y_{m}) + (y_{1} * y_{m + 1}) +  ...  + (y_{n - m} * y_{m + (n - m)}) + (y_{(n - m) + 1} * x_{1}) +  ...  + (y_{(n - m) + m} * x_{m}) + (z * z)

**But not everything is working right, yet.  This is wrong:**

In [144]:
dot_prod_and_more_map = Lambda((a_iter_i_j, b_iter_i_j),
                               Add(Iter(k, Mult(a_k, b_k, k), 
                                        i, j)))

dot_prod_and_more_map: (a_{i},...,a_{j}, b_{i},...,b_{j}) -> ((a_{i} * b_{i} * i) +  ...  + (a_{j} * b_{j} * j))

In [145]:
a_required_len = Equals(Len(a_sub), a_len)
b_required_len = Equals(Len(b_sub), a_len)
dot_prod_and_more_map.apply(*a_sub, *b_sub, 
                            assumptions=[a_required_len,
                                         b_required_len],
                            requirements=requirements)

(x_{1} * y_{1} * 1) +  ...  + (x_{m} * y_{m} * m) + (y_{1} * y_{m + 1} * 1) +  ...  + (y_{n - m} * y_{m + (n - m)} * (n - m)) + (y_{(n - m) + 1} * x_{1} * 1) +  ...  + (y_{(n - m) + m} * x_{m} * m) + (z * z * _y_)

**For a case like this, the requirement should be that the indices match exactly, not just their lengths.**

In [146]:
%end basic_expressions

# Next chapter: <a class="ProveItLink" href="tutorial02_proof_basics.ipynb">Proof Basics</a>

## <a class="ProveItLink" href="tutorial00_introduction.ipynb#contents">Table of Contents</a>